In [1]:
from sklearn.datasets import load_digits
import pandas as pd

In [4]:
digits = load_digits()
dir(digits)

['DESCR', 'data', 'feature_names', 'frame', 'images', 'target', 'target_names']

In [8]:
df = pd.DataFrame(digits.data)
df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0


In [20]:
df['target'] = digits.target
df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0


In [22]:
from sklearn.model_selection import train_test_split
x = df.drop('target', axis=1)
y = df.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [32]:
lr = LogisticRegression(max_iter=1400)
lr.fit(x_train, y_train)
lr.score(x_test, y_test)

0.95

In [33]:
svc = SVC()
svc.fit(x_train, y_train)
svc.score(x_test,y_test)

0.9814814814814815

In [34]:
rf = RandomForestClassifier()
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

0.9777777777777777

### K Fold validation

In [36]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [37]:
for train_index, test_index in kf.split(range(10)):
    print(train_index, test_index)

[4 5 6 7 8 9] [0 1 2 3]
[0 1 2 3 7 8 9] [4 5 6]
[0 1 2 3 4 5 6] [7 8 9]


In [38]:
digits.data

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [40]:
digits.target

array([0, 1, 2, ..., 8, 9, 8])

In [88]:
def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

In [113]:
svc_kfold_score = []
rf_kfold_score = []
lr_kfold_score = []

for train_index, test_index in kf.split(df.drop('target', axis=1), df.target):
    x_train, x_test, y_train, y_test = digits.data[train_index], digits.data[test_index], \
                                       digits.target[train_index], digits.target[test_index]
    svc_kfold_score.append(get_score(SVC(), x_train, x_test, y_train, y_test))
    lr_kfold_score.append(get_score(LogisticRegression(max_iter=2000), x_train, x_test, y_train, y_test))
    rf_kfold_score.append(get_score(RandomForestClassifier(), x_train, x_test, y_train, y_test))

In [114]:
print(svc_kfold_score)
print(lr_kfold_score)
print(rf_kfold_score)

[0.9666110183639399, 0.9816360601001669, 0.9549248747913188]
[0.9282136894824707, 0.9415692821368948, 0.9165275459098498]
[0.9432387312186978, 0.9649415692821369, 0.9198664440734557]


In [115]:
import numpy as np
print(f'SVC : {np.mean(svc_kfold_score)}')
print(f'LR  : {np.mean(lr_kfold_score)}')
print(f'RF  : {np.mean(rf_kfold_score)}')

SVC : 0.9677239844184752
LR  : 0.9287701725097385
RF  : 0.9426822481914301


### Stratified K Fold validation
#### This will ensure each fold test and train has all data of possible values 

In [98]:
from sklearn.model_selection import StratifiedKFold

In [99]:
sFold = StratifiedKFold(n_splits=3)

In [143]:
svc_sFold_score = []
lr_sFold_score = []
rf_sFold_score = []

for train_index, test_index in sFold.split(df.drop('target', axis=1), df.target):
    x_train, x_test, y_train, y_test = digits.data[train_index], digits.data[test_index],\
                                        digits.target[train_index], digits.target[test_index]
    svc_sFold_score.append(get_score(SVC(), x_train, x_test, y_train, y_test))
    lr_sFold_score.append(get_score(LogisticRegression(max_iter=3500), x_train, x_test, y_train, y_test))
    rf_sFold_score.append(get_score(RandomForestClassifier(n_estimators=100), x_train, x_test, y_train, y_test))

In [144]:
print(svc_sFold_score)
print(lr_sFold_score)
print(rf_sFold_score)

[0.9649415692821369, 0.9799666110183639, 0.9649415692821369]
[0.9248747913188647, 0.9382303839732888, 0.9232053422370617]
[0.9432387312186978, 0.9515859766277128, 0.9198664440734557]


In [145]:
print(f'SVC : {np.mean(svc_sFold_score)}')
print(f'LR  : {np.mean(lr_sFold_score)}')
print(f'RF  : {np.mean(rf_sFold_score)}')

SVC : 0.9699499165275459
LR  : 0.9287701725097385
RF  : 0.9382303839732887


## Cross Val Score

In [146]:
from sklearn.model_selection import cross_val_score

In [151]:
cross_val_score(SVC(), x, y, cv=3)

array([0.96494157, 0.97996661, 0.96494157])

In [157]:
cross_val_score(LogisticRegression(max_iter = 3500), x, y, cv=3)

array([0.92487479, 0.93823038, 0.92320534])

In [162]:
cross_val_score(RandomForestClassifier(n_estimators=120), x, y, cv=3)

array([0.93823038, 0.95325543, 0.92487479])